In [60]:
%reset -f

In [61]:
import pandas as pd
df= pd.read_csv('/content/sample_data/eCommerce_purchase.csv')

### **데이터 전처리**

---


1. 중복값 처리



In [62]:
df = df.drop_duplicates()



```
# This is formatted as code
```

2. 결측값 처리

In [63]:
df.describe()
df = df.dropna(subset = ['user_id','price','brand','category_code'])

#결측값 처리 확인
df.isna().sum()

,0
event_time,0
order_id,0
product_id,0
category_id,0
category_code,0
brand,0
price,0
user_id,0


- RFM 관점으로 분석할 것이기 때문에 결측치를 대체하는 것은 분석의 목적을 퇴색시키고 부정확한 결과를 가져올 수 있다고 생각
- 2000000개라는 충분한 데이터, 결측치를 제거한 이후에도 분석하기에 충분한 데이터를 확보할 수 있다는 근거를 가지고 결측치 제거

3. 이상치(Outliers) 처리

In [64]:
#문자형식에서 날짜형식으로 변환
df['event_time'] = pd.to_datetime(df['event_time'])

set_of_year_month = list(set([(x.year, x.month) for x in df['event_time']])) ## 송장날짜에서 연도와 월정보를 가져온다.
for ym in sorted(set_of_year_month, key=lambda x: (x[0], x[1])): ## 연도와 월정보를 연도, 월 순으로 출력
    print(f'{ym[0]}년 {ym[1]}월')

1970년 1월
2020년 1월
2020년 2월
2020년 3월
2020년 4월
2020년 5월
2020년 6월
2020년 7월
2020년 8월
2020년 9월
2020년 10월
2020년 11월


- 날짜에서 1970년이라는 이상치를 발견 -> 제거

In [65]:
df = df[df['event_time'].dt.year != 1970]
print(df['event_time'].describe())

count                                 419390
mean     2020-08-08 03:47:13.900629760+00:00
min                2020-01-05 04:06:18+00:00
25%      2020-07-03 09:37:42.750000128+00:00
50%         2020-08-14 09:12:12.500000+00:00
75%         2020-10-06 14:17:08.500000+00:00
max                2020-11-21 10:10:30+00:00
Name: event_time, dtype: object


In [66]:
df.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39+00:00,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43+00:00,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
5,2020-04-26 08:45:57+00:00,2295716521449619559,1515966223509261697,2.268105e+18,furniture.kitchen.table,maestro,39.33,1.515916e+18
6,2020-04-26 09:33:47+00:00,2295740594749702229,1515966223509104892,2.268105e+18,electronics.smartphone,apple,1387.01,1.515916e+18
10,2020-04-26 14:55:26+00:00,2295902490203259134,2273948311742316796,2.268105e+18,appliances.kitchen.refrigerators,lg,462.94,1.515916e+18


- Category_code는 기기 카테고리.세부제품 형식으로 입력되어 있는 것을 확인

In [67]:
#기기 카테고리
df['category'] = df['category_code'].str.split('.').str[0]

#제품
df['product']=df['category_code'].str.split('.').str[-1]

df.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,category,product
0,2020-04-24 11:50:39+00:00,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18,electronics,tablet
2,2020-04-24 14:37:43+00:00,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18,electronics,headphone
5,2020-04-26 08:45:57+00:00,2295716521449619559,1515966223509261697,2.268105e+18,furniture.kitchen.table,maestro,39.33,1.515916e+18,furniture,table
6,2020-04-26 09:33:47+00:00,2295740594749702229,1515966223509104892,2.268105e+18,electronics.smartphone,apple,1387.01,1.515916e+18,electronics,smartphone
10,2020-04-26 14:55:26+00:00,2295902490203259134,2273948311742316796,2.268105e+18,appliances.kitchen.refrigerators,lg,462.94,1.515916e+18,appliances,refrigerators


- 제한된 데이터의 확보와 세부적인 분석을 위해 해당 데이터를 카테고리, 제품으로 나누어 저장

### **RFM 분석**

---



In [69]:
import pandas as pd
import sqlite3

# SQLite 데이터베이스 연결 (파일 이름은 원하는 대로 설정)
conn = sqlite3.connect('ecommerce.db')

# 데이터프레임을 SQLite 테이블로 저장
df.to_sql('ecommerce_table', conn, if_exists='replace', index=False)

query = "SELECT * FROM ecommerce_table"

# 쿼리 실행 및 결과를 데이터프레임으로 변환
result_df = pd.read_sql(query, conn)

# 결과 확인
print(result_df.head())

                  event_time             order_id           product_id  \
0  2020-04-24 11:50:39+00:00  2294359932054536986  1515966223509089906   
1  2020-04-24 14:37:43+00:00  2294444024058086220  2273948319057183658   
2  2020-04-26 08:45:57+00:00  2295716521449619559  1515966223509261697   
3  2020-04-26 09:33:47+00:00  2295740594749702229  1515966223509104892   
4  2020-04-26 14:55:26+00:00  2295902490203259134  2273948311742316796   

    category_id                     category_code    brand    price  \
0  2.268105e+18                electronics.tablet  samsung   162.01   
1  2.268105e+18       electronics.audio.headphone   huawei    77.52   
2  2.268105e+18           furniture.kitchen.table  maestro    39.33   
3  2.268105e+18            electronics.smartphone    apple  1387.01   
4  2.268105e+18  appliances.kitchen.refrigerators       lg   462.94   

        user_id     category        product  
0  1.515916e+18  electronics         tablet  
1  1.515916e+18  electronics      he

In [71]:
query_getRFMscore = pd.read_sql_query("""

WITH T1 AS (
    SELECT user_id,
           ntile(5) OVER (ORDER BY Recency) AS rfm_Recency,
           ntile(5) OVER (ORDER BY Frequency) AS rfm_Frequency,
           ntile(5) OVER (ORDER BY Monetary) AS rfm_Monetary
    FROM (
        SELECT user_id,
               max(event_time) AS Recency,
               count(user_id) AS Frequency,
               sum(price) AS Monetary
        FROM ecommerce_table
        GROUP BY user_id
    ) AS st1
),

T2 AS (
    SELECT rfm_Recency, rfm_Frequency, rfm_Monetary,
           rfm_Recency || '' || rfm_Frequency || '' || rfm_Monetary AS RFM_score
    FROM T1
)

SELECT rfm_Recency, rfm_Frequency, rfm_Monetary,
       (CASE
            WHEN RFM_score IN ('333', '323') THEN 'VIP Customer'
            WHEN RFM_score IN ('313', '223') THEN 'Loyal Customers'
            WHEN RFM_score IN ('213', '123', '113') THEN 'High-Value Customers'
            WHEN RFM_score IN ('312', '311') THEN 'Prospective Customers'
            WHEN RFM_score IN ('211', '212') THEN 'Risk Customers'
            ELSE 'Churned Customers'
        END) AS Customer_Segment
FROM T2;

"""
,conn)

# 결과 확인
print(query_getRFMscore.head(100))

    rfm_Recency  rfm_Frequency  rfm_Monetary   Customer_Segment
0             1              3             1  Churned Customers
1             1              3             2  Churned Customers
2             1              2             2  Churned Customers
3             1              4             1  Churned Customers
4             1              4             1  Churned Customers
..          ...            ...           ...                ...
95            1              1             1  Churned Customers
96            1              1             2  Churned Customers
97            1              1             1  Churned Customers
98            1              1             1  Churned Customers
99            1              3             2  Churned Customers

[100 rows x 4 columns]


### **코호트 분석**

---



In [112]:
query_cohort = pd.read_sql_query("""

WITH T1 AS (
    SELECT user_id, strftime('%Y-%m-%d', event_time) AS event_time
    FROM ecommerce_table
),

T2 AS (
    SELECT user_id, MIN(event_time) AS cohort_date
    FROM T1
    GROUP BY user_id
),

T3 AS (
    SELECT (julianday(A.event_time) - julianday(B.cohort_date)) / 30 AS cohort_index,
           strftime('%Y-%m-01', B.cohort_date) AS cohort,
           A.user_id
    FROM T1 A
    LEFT JOIN T2 B
    ON A.user_id = B.user_id
    WHERE (julianday(A.event_time) - julianday(B.cohort_date)) / 30 >= 0
)

SELECT cohort, CAST(cohort_index AS INTEGER) AS cohort_index, COUNT(DISTINCT user_id) AS user_count
FROM T3
GROUP BY 1, 2;
"""
,conn)


# 결과 확인
print(query_cohort.head(111))

        cohort  cohort_index  user_count
0   2020-01-01             0        1431
1   2020-01-01             1         426
2   2020-01-01             2         329
3   2020-01-01             3         440
4   2020-01-01             4         579
..         ...           ...         ...
61  2020-09-01             1        1063
62  2020-09-01             2         312
63  2020-10-01             0        5771
64  2020-10-01             1         333
65  2020-11-01             0        2959

[66 rows x 3 columns]


### **리텐션 분석**

In [111]:
query_retetion = pd.read_sql_query("""

    WITH T1 AS (
        SELECT user_id, strftime('%Y-%m-%d', event_time) AS event_time
        FROM ecommerce_table
    ),

    T2 AS (
        SELECT user_id, MIN(event_time) AS cohort_date
        FROM T1
        GROUP BY user_id
    ),

    T3 AS (
        SELECT (julianday(A.event_time) - julianday(B.cohort_date)) / 30 AS cohort_index,
              strftime('%Y-%m-01', B.cohort_date) AS cohort,
              A.user_id
        FROM T1 A
        INNER JOIN T2 B
        ON A.user_id = B.user_id
        WHERE (julianday(A.event_time) - julianday(B.cohort_date)) / 30 >= 0
    ),

    T4 AS (
        SELECT cohort, CAST(cohort_index AS INTEGER) AS cohort_index, COUNT(DISTINCT user_id) AS user_count
        FROM T3
      GROUP BY 1, 2
    ),

    T5 AS (
        SELECT cohort,
              MAX(CASE WHEN cohort_index = 0 THEN user_count ELSE 0 END) AS First_month_Users
        FROM T4
        GROUP BY cohort
    )

    SELECT C.cohort,
          C.cohort_index,
          C.user_count,
          D.First_month_Users,
          ROUND(CAST(C.user_count AS FLOAT) / D.First_month_Users * 100, 2) AS retention_rate
    FROM T4 C
    INNER JOIN T5 D
    ON C.cohort = D.cohort
    ORDER BY 1, 2;

"""
,conn)


# 결과 확인
print(query_retetion.head(111))

        cohort  cohort_index  user_count  First_month_Users  retention_rate
0   2020-01-01             0        1431               1431          100.00
1   2020-01-01             1         426               1431           29.77
2   2020-01-01             2         329               1431           22.99
3   2020-01-01             3         440               1431           30.75
4   2020-01-01             4         579               1431           40.46
..         ...           ...         ...                ...             ...
61  2020-09-01             1        1063               8552           12.43
62  2020-09-01             2         312               8552            3.65
63  2020-10-01             0        5771               5771          100.00
64  2020-10-01             1         333               5771            5.77
65  2020-11-01             0        2959               2959          100.00

[66 rows x 5 columns]
